In [4]:
# Import modules
import numpy as np
import pandas as pd


In [5]:
# Load csv & display some basic details
properties = pd.read_csv('properties.csv', header=0)
#display(properties.head())
#display(properties.tail())
display(properties.shape)
#display(properties.columns.tolist())


(11992, 22)

In [12]:
# Clean boolean & NaN values as binary
# Iterate through rows, then columns
for ind , row in properties.iterrows():
    for column in properties.columns:
        
# Replace booleans with 0 or 1      
        if row[column] == False:
            properties.at[ind , column] = 0
        elif row[column] == True:
            properties.at[ind , column] = 1
            
           
# If NaN is inplace, replace with a 0. Assume that if the data is not entered, the property does not have the attribute.           
properties.fillna(0, inplace=True)

# Delete type_of_sale since the entire column's data is the same.
try:
    properties = properties.drop('type of sale', axis=1)
except:
    pass

# Clean kitchen data
for index, row in properties.iterrows():
    if row['fully equipped kitchen'] == 'usa_uninstalled' or row['fully equipped kitchen'] == 'not_installed' or row['fully equipped kitchen'] == '0': 
        properties.at[index , 'fully equipped kitchen'] = 0
    elif row['fully equipped kitchen'] != 0:
        properties.at[index , 'fully equipped kitchen'] = 1

# Clean rennovation data
for index, row in properties.iterrows():
    if row['state of the building'] == 'to_be_done_up' or row['state of the building'] == 'to_renovate' or row['state of the building'] == 'to_restore':
        properties.at[index , 'state of the building'] = 'needs renovating'
    elif row['state of the building'] == 0:
        properties.at[index , 'state of the building'] = str('no data')
    else:
        pass

# Swap province and locality columns
prop_colummns = properties.columns.tolist()
prop_colummns[3] , prop_colummns[4] = prop_colummns[4] , prop_colummns[3]
properties = properties[prop_colummns]
 
# Sort rows by postcode
properties = properties.sort_values(by='postalCode')
pd.set_option('display.max_columns', 100)  
pd.set_option('display.max_rows', 20) 

# Confirm there are no duplicated rows
print(properties.loc[properties.duplicated(), :])

"""# Replace string-values with values
for index, row in properties.iterrows():
    for column in properties.columns:
        value = row[column]
        if type(value) == str and value.isdigit():
            properties.at[index, column] = int(value)
        if type(value) == float:
            properties.at[index, column] = int(value)"""

# Change value column type to values
for column in properties.columns[6:20]:
    properties[column] = properties[column].astype(int)


properties

Empty DataFrame
Columns: [id, type of property, subtype of property, province, locality, postalCode, price, number of bedrooms, living area, fully equipped kitchen, furnished, open fire, terrace, terrace area, garden, garden area, total property area, total land area, number of facades, swimming pool, state of the building]
Index: []


,id,type of property,subtype of property,province,locality,postalCode,price,number of bedrooms,living area,fully equipped kitchen,furnished,open fire,terrace,terrace area,garden,garden area,total property area,total land area,number of facades,swimming pool,state of the building
1114,10640355,APARTMENT,APARTMENT,Brussels,Bruxelles,1000,310000,1,35,1,0,0,0,0,0,0,69,0,2,0,just_renovated
1242,10449929,APARTMENT,APARTMENT,Brussels,Bruxelles,1000,550000,3,0,1,0,1,1,10,0,0,108,0,0,0,good
2608,10462628,APARTMENT,APARTMENT,Brussels,BRUXELLES,1000,393000,2,0,1,0,1,1,8,0,0,96,0,2,0,good
1171,10183530,APARTMENT,APARTMENT,Brussels,Brussels City,1000,375000,2,0,1,1,0,1,25,0,0,95,0,2,0,good
1177,10070305,APARTMENT,FLAT_STUDIO,Brussels,Bruxelles,1000,385000,0,0,1,1,0,0,0,0,0,55,0,2,0,as_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8069,10563918,APARTMENT,APARTMENT,East Flanders,Maldegem,9990,240000,1,20,1,0,0,1,8,1,0,68,0,0,0,no data
3004,10648690,HOUSE,HOUSE,East Flanders,Adegem,9991,399000,4,0,1,0,1,1,0,0,0,206,5239,4,0,needs renovating
11523,10658294,HOUSE,HOUSE,East Flanders,Maldegem,9991,345000,3,0,0,0,0,0,0,1,0,175,766,4,0,needs renovating
10388,10661746,HOUSE,HOUSE,East Flanders,Maldegem,9991,345000,3,0,0,0,0,0,0,1,0,175,766,4,0,needs renovating


In [17]:
# Check distribution of unique values in each column
for column in properties.columns:
    column_data = properties[column]
    value_counts = column_data.value_counts()
    print(value_counts , '\n')

id
10640355    1
10547722    1
10669714    1
10281099    1
10678165    1
           ..
10619498    1
10546424    1
8822156     1
10628821    1
10634589    1
Name: count, Length: 11992, dtype: int64 

type of property
APARTMENT    5997
HOUSE        5995
Name: count, dtype: int64 

subtype of property
APARTMENT          4548
HOUSE              4088
VILLA               801
DUPLEX              427
APARTMENT_BLOCK     314
                   ... 
FARMHOUSE            20
KOT                  18
CHALET               14
OTHER_PROPERTY       11
CASTLE               10
Name: count, Length: 23, dtype: int64 

province
Brussels           2470
West Flanders      2247
East Flanders      1347
Antwerp            1274
Liège              1192
Flemish Brabant    1169
Hainaut             717
Walloon Brabant     687
Luxembourg          421
Namur               334
Limburg             133
0                     1
Name: count, dtype: int64 

locality
Uccle                379
Knokke-Heist         320
Ixelles    